In [60]:
import pandas as pd
import os
%matplotlib inline
import seaborn

In [103]:
print(os.getcwd())
os.chdir('./electricity_data_nctu/03report/')
os.listdir()

/Users/ah-bau/github_repo


['.ipynb_checkpoints', '01_資料欄位前處理-Copy1.ipynb', '館舍報告-Pipeline-科三為例.ipynb']

In [104]:
# Global variables
CostRate = 2.8 # 一度電多少錢（估算，實際還要再確認）

In [119]:
file = 'teach_p2.csv'
path = '../01data/106B_P1_P4/{}'.format(file)
df = pd.read_csv(path)

In [120]:
print('Old names: ',df.columns)
print('Reassign Column names')
df.columns = ['DateTime', 'Area', 'Building', 'Loop', 'Type', 'Usage']
print('New names: ',df.columns)
df.head()

Old names:  Index(['時  間', '區 域', '館 舍 別', '迴 路 別', '用電別', '用電量(單位:度)'], dtype='object')
Reassign Column names
New names:  Index(['DateTime', 'Area', 'Building', 'Loop', 'Type', 'Usage'], dtype='object')


,DateTime,Area,Building,Loop,Type,Usage
0,2018/03/12 00:00,北區,工程一館,主迴路,教學,15.38
1,2018/03/12 01:00,北區,工程一館,主迴路,教學,15.63
2,2018/03/12 02:00,北區,工程一館,主迴路,教學,15.00
3,2018/03/12 03:00,北區,工程一館,主迴路,教學,15.00
4,2018/03/12 04:00,北區,工程一館,主迴路,教學,15.25


In [121]:
def clear_white_space(df):
    print("Before: ", df.Building.unique())
    for y in df.select_dtypes(include = 'object').columns:
        df[y] = df[y].str.strip()
    print('After: ', df.Building.unique())
clear_white_space(df)

Before:  ['工程一館                ' '工程二館                ' '管理一館                '
 '電資大樓                ' '光電大樓                ' '基礎大樓                '
 '科學一館                ' '科學二館                ' '環保大樓                '
 '人社一館                ' '人社二館                ' '人社三館                '
 '工程三館                ' '工程四館                ' '工程五館                '
 '工程六館                ' '管理二館                ' '綜合一館                ']
After:  ['工程一館' '工程二館' '管理一館' '電資大樓' '光電大樓' '基礎大樓' '科學一館' '科學二館' '環保大樓' '人社一館'
 '人社二館' '人社三館' '工程三館' '工程四館' '工程五館' '工程六館' '管理二館' '綜合一館']


In [122]:
# 用自訂函式去理解DF中每個欄位有哪幾種類別的值
print_limit = 100
def understand(df):
    for col_name in df.columns:
        print ('>>> "{}" has {} unique values:'.format(col_name, len(df[col_name].unique())))
        if(len(df[col_name].unique())< print_limit):
            print(df[col_name].unique())
        else:
            print('The value is more than {} so wont be list here :P'.format(print_limit))
        print('\n')


In [123]:
understand(df)

>>> "DateTime" has 840 unique values:
The value is more than 100 so wont be list here :P


>>> "Area" has 2 unique values:
['北區' '南區']


>>> "Building" has 18 unique values:
['工程一館' '工程二館' '管理一館' '電資大樓' '光電大樓' '基礎大樓' '科學一館' '科學二館' '環保大樓' '人社一館'
 '人社二館' '人社三館' '工程三館' '工程四館' '工程五館' '工程六館' '管理二館' '綜合一館']


>>> "Loop" has 11 unique values:
['主迴路' '空調' '網球場動力' '網球場燈力' '空大MP動力' '空大ML燈力' '棒球場照明' '餐亭' '管科ML燈力' '管科插座'
 '管科MP動力']


>>> "Type" has 3 unique values:
['教學' '行政' '外租']


>>> "Usage" has 3307 unique values:
The value is more than 100 so wont be list here :P




### 原始資料說明 - 2
- index: ['﻿時  間', '區 域', '館 舍 別', '迴 路 別', '用電別', '用電量(單位:度)']

### 人工資料前處理：改變欄位名稱
#### 改變的原因
- 原始的資料有空白在字串裡，會造成一些麻煩
- 英文的名稱，可以在Pandas寫Code時較為方便(才能用 df.Usage的指令)

#### 改了什麼
- 我用Number(Excel)的尋找取代功能，把資料轉成英文，以下括弧表示原始資料的命名
    - 區域：southArea(南區), northArea(北區)
    - 迴 路 別: mainLoop（主迴路）, familyMart（1F便利超商）, club（社團）, socialHall（閱覽交誼廳）, studentHall（學生交誼廳）, streetLight(環校路燈),
    - 館舍： dorm_13（學生十三舍）,(7,8,9,10,11,12 以此類推), dorm_graduate_2（研二）, dorm_87（學生七舍＿八舍）, dorm_13_graduate_1(11與研一), dorm_chu（竹軒）
    - 用電別：rent（外租）, dorm（宿舍）, admin(行政)

In [124]:
dic_replace = {
    'Area' : {'北區':'N','南區':'S'},
    'Building' : {
        '基礎大樓':"SB3",
        '科學二館':"SB2",
        '科學一館':"SB1",
        '學生十三舍': "D13", 
        '學生十二舍': 'D12',
        '學生十一舍': 'D11',
        '學生十舍': 'D10',
        '學生九舍': 'D9',
        '學生八舍': 'D8',
        '學生七舍': 'D7',
        '學生七舍_八舍':'D87',
        '竹軒宿舍': 'Df1',
        '女二舍': 'Df2',
        '研究生一舍': 'Dgd1',
        '研究生二舍': 'Dgd2',
        '學生十一舍_研一舍':'D11_grad1',
        '管理一館':'MB1',
        '管理二館':'MB2',
        '工程一館': 'EB1',
        '工程二館': 'EB2',
        '工程三館': 'EB3',
        '工程四館': 'EB4',
        '工程五館': 'EB5',
        '工程六館': 'EB6',
        '人社一館': 'HS1',
        '人社二館': 'HS2',
        '交映樓': 'CPT',
        '土木結構': 'CElab',
        '中正堂':'CKS',
        '光電大樓':'TKB',
        '第二餐廳': 'Canteen2',
        '服務大樓':'Service',
        '行政大樓':'Administration',
        '環保大樓':'Environmental',
        '材料實驗':'Material',
        '綜合一館':'Assembly1',
        '綜合球館':'Sport',
        '游泳館':'Swim',
        '體育館':'Gym',
        '機車Ｈ棚':'ShedH',
        '機車Ａ棚':'ShedA',
        '機車Ｄ棚':'ShedD',
        '工程一館中繼':'PowerSatatiopn_EB1',
        '人社二館中繼':'PowerSatatiopn_HS2',
        '南區總電站':'PowerSatatiopn_main_south',
        '防災中心':'Safty',
        '污水場':'Wastewater',
        '電資大樓': 'ElecInfo',
        '資訊館(南區)':'Info_south',
        '資訊館(北區)':'Info_north',
        '活動中心': 'Activity',
        '圖資中心': 'Library',
        '第一招待所': 'GuestHouse'
                 },
    'Type' : {'教學': 'edu', '宿舍': 'dorm', '行政':'admin', '外租':'rent','電站':'power'},
    'Loop' : {'主迴路': 'main'}
              }
# ,'Building': {'學生十舍':'d10', '學生十三舍':'d13', }
df = df.replace(dic_replace)

In [125]:
df.Building.unique()

array(['EB1', 'EB2', 'MB1', 'ElecInfo', 'TKB', 'SB3', 'SB1', 'SB2',
       'Environmental', 'HS1', 'HS2', '人社三館', 'EB3', 'EB4', 'EB5', 'EB6',
       'MB2', 'Assembly1'], dtype=object)

In [126]:
understand(df)

>>> "DateTime" has 840 unique values:
The value is more than 100 so wont be list here :P


>>> "Area" has 2 unique values:
['N' 'S']


>>> "Building" has 18 unique values:
['EB1' 'EB2' 'MB1' 'ElecInfo' 'TKB' 'SB3' 'SB1' 'SB2' 'Environmental'
 'HS1' 'HS2' '人社三館' 'EB3' 'EB4' 'EB5' 'EB6' 'MB2' 'Assembly1']


>>> "Loop" has 11 unique values:
['main' '空調' '網球場動力' '網球場燈力' '空大MP動力' '空大ML燈力' '棒球場照明' '餐亭' '管科ML燈力'
 '管科插座' '管科MP動力']


>>> "Type" has 3 unique values:
['edu' 'admin' 'rent']


>>> "Usage" has 3307 unique values:
The value is more than 100 so wont be list here :P




In [127]:
# os.chdir(path[:path.rfind('/')])

In [128]:
def generate_date_time_columns(df):
    if('DateTime' in df.columns):
            df['Date'] = df.DateTime.str[:10]
            df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
            df['Weekday'] = df['Date'].dt.dayofweek + 1
            df['DayName'] = df['Date'].dt.strftime("%A")
            df['Time'] = df.DateTime.str[11:]
            df['Week'] = df['Date'].dt.week
            print('Columns #Date, #Time, #Weekday, #Week has been generated from DateTime and added to DataFrame')
    else:
        print('DateTime not in column. Please rename columns name.')
    return df

In [129]:
# # cost_rate：一度電大概多少錢
# def generate_cost_columns(df, cost_rate):
#     df['Cost({})'.format(CostRate)] = df.Usage * cost_rate
#     print('Column "Cost" had been generated!')
#     return df

In [130]:
df = generate_date_time_columns(df)
# df = generate_cost_columns(df, CostRate)
df.head()

Columns #Date, #Time, #Weekday, #Week has been generated from DateTime and added to DataFrame


,DateTime,Area,Building,Loop,Type,Usage,Date,Weekday,DayName,Time,Week
0,2018/03/12 00:00,N,EB1,main,edu,15.38,2018-03-12,1,Monday,00:00,11
1,2018/03/12 01:00,N,EB1,main,edu,15.63,2018-03-12,1,Monday,01:00,11
2,2018/03/12 02:00,N,EB1,main,edu,15.00,2018-03-12,1,Monday,02:00,11
3,2018/03/12 03:00,N,EB1,main,edu,15.00,2018-03-12,1,Monday,03:00,11
4,2018/03/12 04:00,N,EB1,main,edu,15.25,2018-03-12,1,Monday,04:00,11


In [141]:
os.getcwd()
os.listdir()
os.chdir('./01data/106B_P1_P4/')

In [143]:
df.to_csv('+'+file, encoding='utf-8')
print('存檔，作業完成！')

存檔，作業完成！


In [144]:
os.listdir()

['+teach_p1.csv', '+teach_p2.csv', 'teach_p1.csv', 'teach_p2.csv']